<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train7_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train7 = pd.read_csv(r'/content/train_data7.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train7['Target'] = train7['Target'].replace(['Others'],'Invalid')


In [ ]:
train7['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train7_Xtfidf = tfidf_vect.fit_transform(train7['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train7_Xtfidf,train7['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6714601769911505
                  precision    recall  f1-score   support

        Analysis       0.54      0.34      0.42       130
      Conclusion       0.83      0.39      0.53        49
           Facts       0.72      0.89      0.80       475
         Invalid       0.52      0.58      0.55       128
           Issue       0.59      0.45      0.51        53
Rule/Law/Holding       0.79      0.33      0.47        69

        accuracy                           0.67       904
       macro avg       0.67      0.50      0.55       904
    weighted avg       0.67      0.67      0.65       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 0, 3, 2, 3, 2, 3, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 0, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2]
[0.9980236451501517, 0.9978168643309786, 0.9931749192692562, 0.9817795309700634, 0.9986015765099451, 0.9989238194973068, 0.9979911072400218, 0.9891185379223937, 0.9910276198062397, 0.9982649274301193, 0.9908158123378119, 0.9947634063008488, 0.9981795315528698, 0.991671802351068, 0.9975518016981306, 0.9971425556712974, 0.9980104752406912, 0.9984868799315744, 0.9984495676226276, 0.9937624579760492, 0.9970819200090758, 0.9984918463873549, 0.9933027078606442, 0.9992205494364869, 0.9983572441551474, 0.9953112071156301, 0.9969729917245617, 0.9971679313691909, 0.9830825341240141, 0.9906129524047323, 0.9833324680830593, 0.9971948517591867, 0.9973297646670641, 0.9878959932818262, 0.9929193072136521, 0.9980384552355904, 0.9915150061636184, 0.9960261390665776, 0.9983880549381816, 0.9912795155974778, 0.9967121412563755,

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
3,2199,proper instruction court murder malice murder ...,Facts
4,79,eligible leaves renew contacts family,Facts


In [ ]:
frame_1 = [train7,unlabel_1]
train7_1 = pd.concat(frame_1)
len(train7_1)

3536

In [ ]:
x_train_1 = tfidf_vect.transform(train7_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train7_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.653954802259887
                  precision    recall  f1-score   support

        Analysis       0.48      0.40      0.43       103
      Conclusion       0.67      0.45      0.54        44
           Facts       0.71      0.88      0.78       347
         Invalid       0.61      0.56      0.58       117
           Issue       0.67      0.40      0.50        45
Rule/Law/Holding       0.52      0.29      0.37        52

        accuracy                           0.65       708
       macro avg       0.61      0.50      0.53       708
    weighted avg       0.64      0.65      0.64       708



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 3, 3, 2, 4, 3, 2, 2, 2, 2, 2, 5, 3, 5, 2]
[0.9979636871187383, 0.9990399797613445, 0.98792223540388, 0.9963585199103424, 0.9979263216176896, 0.9986754305735424, 0.9981528305890931, 0.9989927428291135, 0.9931555944961715, 0.9935140524902699, 0.9981340048142912, 0.9842886004646645, 0.9923210220311766, 0.996033195801861, 0.995275787431389, 0.9977174716455786, 0.9958487742666499, 0.9988459415246745, 0.9990274110438434, 0.9913480954727685, 0.9982258799288284, 0.9992636960608776, 0.9961137587059478, 0.9969459187074294, 0.9954400675557507, 0.9951307682894341, 0.9986135690754928, 0.9953229858522868, 0.9988697120509845, 0.9988252194108194, 0.9810710353079525, 0.9972476455822377, 0.9985796456502171, 0.9846565149422851, 0.997522482908672, 0.994249804136901, 0.9889951628344428, 0.991336011703071, 0.9976608600690771, 0.9955702521916308, 0.99824935443596, 0.998877289836907, 0.9816300259429

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train7_1,unlabel_2]
train7_2 = pd.concat(frame_2)
len(train7_2)

3587

In [ ]:
x_train_2 = tfidf_vect.transform(train7_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train7_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6480908152734778
                  precision    recall  f1-score   support

        Analysis       0.51      0.29      0.37       151
      Conclusion       0.66      0.37      0.47        52
           Facts       0.69      0.90      0.78       505
         Invalid       0.52      0.55      0.54       146
           Issue       0.79      0.27      0.41        55
Rule/Law/Holding       0.59      0.28      0.38        60

        accuracy                           0.65       969
       macro avg       0.63      0.44      0.49       969
    weighted avg       0.64      0.65      0.62       969



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 4, 2, 2, 2, 2, 2]
[0.9974796554454781, 0.9843482177287779, 0.9943961827382362, 0.9984972137668199, 0.9972592549991531, 0.9945369244485046, 0.9986458768814309, 0.9984236103224867, 0.9981750610363805, 0.9986019642561399, 0.9851716464105699, 0.9987178259585827, 0.9986796766974383, 0.9982945356499141, 0.9950211488791614, 0.9981903529578132, 0.994307501646481, 0.9906151754604526, 0.9947086872119991, 0.9981563925548753, 0.998036209308284, 0.9899362064271241, 0.9983935412962471, 0.9908536129250556, 0.9980790377024791, 0.9977796754935704, 0.9983150159633919, 0.9949683685580525, 0.9895417561022238, 0.9952238035314456, 0.9979149948185632, 0.9823643216276299, 0.9985646202439189, 0.9967011038602032, 0.9980406120189032, 0.9983062294521, 0.9977164416858786, 0.9858601883576723, 0.9985556187985093, 0.9915392014282962, 0.998721463885

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train7_2,unlabel_3]
train7_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train7_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train7_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6506849315068494
                  precision    recall  f1-score   support

        Analysis       0.50      0.39      0.44        99
      Conclusion       0.90      0.43      0.58        42
           Facts       0.70      0.88      0.78       368
         Invalid       0.53      0.54      0.53       111
           Issue       0.72      0.36      0.48        50
Rule/Law/Holding       0.55      0.27      0.36        60

        accuracy                           0.65       730
       macro avg       0.65      0.48      0.53       730
    weighted avg       0.65      0.65      0.63       730



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 0, 3, 2, 5, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 0, 2, 2, 3, 3, 0, 2, 3, 2, 5, 3, 0, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9985814727714338, 0.9825364974562459, 0.9955555292352852, 0.9848078652644006, 0.9975198823737208, 0.9951827571258971, 0.9984703647709179, 0.9902250107522863, 0.9907702699510343, 0.9987480114587298, 0.9988042665112633, 0.9853428111008043, 0.9936894758588082, 0.9977034341696762, 0.9983101723513643, 0.9951452908757472, 0.9986981680086234, 0.9982381185210867, 0.9986931083318316, 0.9982974161633625, 0.9901288619501026, 0.9879896209995884, 0.9986131267197618, 0.9985017735223376, 0.9987264175350211, 0.9963281435762015, 0.998010139021793, 0.998148296974431, 0.9986775138661831, 0.9982283490381196, 0.9984238640921496, 0.9950275599518433, 0.9951219277819667, 0.9945861998137466, 0.9981917899344034, 0.9847350417635354, 0.9865389315035171, 0.9883330909828086, 0.9974148894358585, 0.98367735622477

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train7_3,unlabel_4]
train7_4 = pd.concat(frame_4)
len(train7_4)
x_train_4 = tfidf_vect.transform(train7_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train7_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6633466135458167
                  precision    recall  f1-score   support

        Analysis       0.52      0.33      0.40       144
      Conclusion       0.68      0.37      0.48        46
           Facts       0.71      0.89      0.79       528
         Invalid       0.55      0.55      0.55       153
           Issue       0.73      0.36      0.48        66
Rule/Law/Holding       0.67      0.36      0.47        67

        accuracy                           0.66      1004
       macro avg       0.64      0.48      0.53      1004
    weighted avg       0.65      0.66      0.64      1004



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 0, 2, 3, 3, 2, 5, 3, 3, 4, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 4, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 5, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 2, 3, 2]
[0.9858415603516953, 0.9989204061902155, 0.9989126630901536, 0.9852668403408573, 0.9986506538168786, 0.9958777958788039, 0.9935185089417905, 0.9979892993306755, 0.9936909643806824, 0.9987507637080356, 0.9988032002324907, 0.9946177421105502, 0.9980834172956887, 0.9987876156491313, 0.9973163233160264, 0.9861929059620128, 0.993449387380948, 0.9936513439894427, 0.9877874923977161, 0.9984749805684606, 0.9988753213717223, 0.9943959253701641, 0.9952496109701232, 0.9958200446072699, 0.9982458113432762, 0.9975459133837259, 0.9830933883056147, 0.9965801902745722, 0.9971246399188488, 0.9984904157516745, 0.9987753456612671, 0.9985890858992378, 0.9989126630901536, 0.997677539206127, 0.9972247089350262, 0.9953554766617679, 0.9985167194666255, 0.9979922980991004, 0.9984749805684606, 0.9960814120628065, 0.995

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train7_4,unlabel_5]
train7_5 = pd.concat(frame_5)
len(train7_5)
x_train_5 = tfidf_vect.transform(train7_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train7_5['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6702544031311155
                  precision    recall  f1-score   support

        Analysis       0.50      0.32      0.39       135
      Conclusion       0.72      0.47      0.57        55
           Facts       0.71      0.90      0.80       530
         Invalid       0.58      0.57      0.58       170
           Issue       0.70      0.38      0.49        61
Rule/Law/Holding       0.57      0.28      0.38        71

        accuracy                           0.67      1022
       macro avg       0.63      0.49      0.53      1022
    weighted avg       0.65      0.67      0.65      1022



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7562380038387716


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7753853281411999, 0.6440787612659502, 0.6901529341130282, None)
